# data fetch
workdir=~/workspace/mNGS/data
cd $workdir
# hg19
wget -c ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz
tar -zxvf chromFa.tar.gz
cat chr*.fa > hg19.fa

# zymo
prefetch SRR27174691
fastq-dump --gzip --split-files SRR27174691.sra

# in silico seq simulation using human genome
seed=202406
wgsim -d 400 -N 18000000 -1 100 -2 100 -S $seed hg19.fa human.R1.fq human.R2.fastq
# `$seed` is a random value which usually use date. so here use 202406

zymo=${workdir}/sra/SRR27174691/SRR27174691
seqtk sample -s $seed ${zymo}_1.fastq.gz 2000000 > zymo.sample.R1.fq
seqtk sample -s $seed ${zymo}_2.fastq.gz 2000000 > zymo.sample.R2.fq


In [2]:
! WORKDIR=~/workspace/mNGS/data
! REFDIR=${WORKDIR}/reference
! repFile=${WORKDIR}/simdata_bacteria/report
! DB=kraken2_db

SyntaxError: invalid syntax (1302630448.py, line 2)

In [ ]:
! for i in `ls simdata_bacteria/*host_rm*`
do
echo $i
    kraken2 --db ${DB} --threads 24 ${WORKDIR}/${i} \
    --output ${repFile}/${i%.host*}_output.kraken --report ${repFile}/${i%.host*}.kreport
done

In [ ]:
kraken2 --db kraken2_db --threads 24 --paired simdata_bacteria/merge_all.R1.fastq simdata_bacteria/merge_all.R2.fastq --output merge.output --report merge.clean.kreport 

In [ ]:
bracken  -i  merge.clean.kreport -d $DB -o  merge.kreport.bracken -w merge.bracken.report -t 8

In [5]:
# 指定要查找的分类号列表
classification_ids = ['3122075', '190721', '119219', '457921']

# 打开Kraken报告文件
with open('/home/Lchen/workspace/mNGS/data/blo-20240725_timeopti/upload/report/K450014659_L01_9292.host_rm_k2db.kraken', 'r') as file:
    for line in file:
        # 去除行尾的换行符并分割行
        parts = line.strip().split()
        
        # 确保行有足够的部分
        if len(parts) < 3:
            continue  # 如果行不完整，跳过
        
        # 第二列是reads标识，第三列开始是分类信息
        reads_id = parts[1]
        
        # 从第三列开始，检查每个部分是否包含指定的分类号
        for part in parts[2:]:
            # 分类信息格式为："分类号:覆盖度 时间"
            if ':' in part:
                classification, _, _ = part.partition(':')
                if classification in classification_ids:
                    print(f'Found classification ID {classification}, reads ID: {reads_id}')
                    break  # 找到匹配项后，不再继续检查该行的其他部分

Found classification ID 3122075, reads ID: K450014659L1C001R03901109040
Found classification ID 3122075, reads ID: K450014659L1C001R04900698877
Found classification ID 119219, reads ID: K450014659L1C001R04901266866
Found classification ID 3122075, reads ID: K450014659L1C001R05301189174
Found classification ID 190721, reads ID: K450014659L1C002R01401168011
Found classification ID 119219, reads ID: K450014659L1C002R01900782176
Found classification ID 190721, reads ID: K450014659L1C002R02900316607
Found classification ID 119219, reads ID: K450014659L1C002R03001418364
Found classification ID 119219, reads ID: K450014659L1C002R03301091073
Found classification ID 119219, reads ID: K450014659L1C002R03400963522
Found classification ID 119219, reads ID: K450014659L1C002R03501222687
Found classification ID 119219, reads ID: K450014659L1C002R03701398624
Found classification ID 119219, reads ID: K450014659L1C002R04101087823
Found classification ID 119219, reads ID: K450014659L1C002R05200117058
Fou

In [6]:

classification_ids = ['3122075', '190721', '119219', '457921']

extracted_reads_ids = []
with open('/home/Lchen/workspace/mNGS/data/blo-20240725_timeopti/upload/report/K450014659_L01_9292.host_rm_k2db.kraken', 'r') as kraken_file, open('reads_ids.txt', 'w') as ids_file:
    for line in kraken_file:
        parts = line.strip().split()
        if len(parts) >= 3:
            reads_id = parts[1]
            for part in parts[2:]:
                if ':' in part:
                    classification, _, _ = part.partition(':')
                    if classification in classification_ids:
                        extracted_reads_ids.append(reads_id)
                        ids_file.write(reads_id + '\n')
                        break

print(f'Total reads IDs extracted: {len(extracted_reads_ids)}')

Total reads IDs extracted: 19


In [7]:
! seqtk subseq /home/Lchen/workspace/mNGS/data/blo-20240725_timeopti/upload/K450014659_L01_9292.host_rm.R1.fastq.gz reads_ids.txt > extracted_reads.R1.fastq